# Exemplo de programação de operações em Flow Shop

Programa que lê um conjunto de tempos de processamento de tarefas em um ambiente Flow Shop e uma sequencia. Ele deve retornar o makespan.

Importação das bibliotecas 

In [14]:

import numpy as np
from prettytable import PrettyTable
import plotly.io as pio
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import plotly.graph_objs as go# for Gantt
from plotly.offline import download_plotlyjs, init_notebook_mode,  plot #for Gantt
import random
import copy 
import time
import timeit
from collections import defaultdict
import datetime                         #for Gantt (time formatting) #duration is in days
import plotly                           #for Gantt (full package)
import plotly.figure_factory as ff      #for Gantt
import chart_studio.plotly as py  



Leitura dos dados

In [15]:
conteudo = []
fob = open("dadosFlowshop.txt", "r") # esta instância contém 5 tarefas e 3 máquinas
for linha in fob.readlines():
    #print("A linha é: ",linha)
    conteudo.append([int(i) for i in linha.split()])
fob.close()

A linha é:  12 13 45 

A linha é:  15 10 14

A linha é:  5 20 5 

A linha é:  6 18 18 

A linha é:  17 5 22



Matriz de tempos de processamento

In [16]:

tempos_Proc = np.array(conteudo) #Lendo matriz. Desse jeito, tempos_Proc[n][m]
print(tempos_Proc)

[[12 13 45]
 [15 10 14]
 [ 5 20  5]
 [ 6 18 18]
 [17  5 22]]



Declaração dos conjuntos

In [17]:
M =  len(tempos_Proc[0]) # número de jobs
N =  len(tempos_Proc) # número de máquinas

print(f' O número de jobs a serem sequenciados é {N} e o número de máquinas é {M}')

 O número de jobs a serem sequenciados é 5 e o número de máquinas é 3


Sequencia a ser analisada:

In [18]:
seq = [2, 1, 3, 0, 4]

Cálculo do makespan:

In [19]:
completionTimes = np.zeros((N, M)) # Criação da matrix de Compretion Times

i = seq[0]
completionTimes[i][0] = tempos_Proc[i][0]
for m in range(1,M):
    i = seq[0]
    completionTimes[i][m] = completionTimes[i][m-1] + tempos_Proc[i][m]
for i in range(1, len(seq)):
    atual = seq[i]
    anterior = seq[i-1]
    completionTimes[atual][0] = completionTimes[anterior][0] + tempos_Proc[atual][0]
    for m in range(1, M):
        completionTimes[atual][m] = max(completionTimes[atual][m-1],completionTimes[anterior][m])  + tempos_Proc[atual][m]

Impressão dos resultados em formato de tabela

In [20]:
pt = PrettyTable(["M1", "M2", "M3"])
for i in completionTimes:
    pt.add_row(i)
print(pt)
print("Makespan =",np.max(completionTimes))

+------+------+-------+
|  M1  |  M2  |   M3  |
+------+------+-------+
| 38.0 | 66.0 | 116.0 |
| 20.0 | 35.0 |  49.0 |
| 5.0  | 25.0 |  30.0 |
| 26.0 | 53.0 |  71.0 |
| 55.0 | 71.0 | 138.0 |
+------+------+-------+
Makespan = 138.0


Computação dos parâmetros para Construção do Gráfico de Gannt

In [21]:
seq_otima = [] # Como neste caso estamos avaliando somente uma sequência ela será considerada como ótima (apesar de não ser... analisar Gannt)
# Este exemplo é didático, portanto todas as sequencias devem ser avaliadaas para a descoberta no menor Makespan

for i in seq:
    seq_otima.append(tempos_Proc[i])

print(seq_otima)

[array([ 5, 20,  5]), array([15, 10, 14]), array([ 6, 18, 18]), array([12, 13, 45]), array([17,  5, 22])]


In [22]:
#Construção da matriz de tempo de início das tarefas

inicio = np.zeros((N,M))


for i in range (N):
  for j in range(M):
    if i==0 and j==0:
      inicio[0][0]=0
    elif i>0 and j==0:
      inicio[i][j] =inicio[i-1][j+1]   
    elif i==N-1:
      inicio[i][j] = inicio[i][j-1]+ seq_otima[i][j-1]
    else:
      if  j<M-1:
        aux_comparacao = inicio[i][j-1] + seq_otima[i][j-1] 
        if aux_comparacao < inicio[i-1][j+1]:
          #print(aux_comparacao, "<" , inicio[i-1][j+1] )
          inicio[i][j] = inicio[i-1][j+1]
        else:  
          inicio[i][j] =inicio[i][j-1]+ seq_otima[i][j-1] 
      else:  
        if i==0:
          inicio[i][j] = inicio[i][j-1] + seq_otima[i][j-1]
        else:
          aux_comparacao = inicio[i][j-1] + seq_otima[i][j-1]
          if aux_comparacao > inicio[i][j]:
            inicio[i][j] = aux_comparacao
          else:
            inicio[i][j] =inicio[i][j-1]+ seq_otima[i][j-1]

print("Inicio das tarefas")   
print(inicio)

Inicio das tarefas
[[ 0.  5. 25.]
 [ 5. 25. 35.]
 [25. 35. 53.]
 [35. 53. 66.]
 [53. 70. 75.]]


In [23]:
#Construção da matriz de tempo de término das tarefas

termino = np.zeros((N,M))

for i in range(N):
  for j in range(M):
    termino[i][j] = seq_otima[i][j] + inicio[i][j]

print("Término")
print(termino)

Término
[[  5.  25.  30.]
 [ 20.  35.  49.]
 [ 31.  53.  71.]
 [ 47.  66. 111.]
 [ 70.  75.  97.]]


In [25]:
#Impressao do Gantt

df = []
colors =[]
for i in range(N):
    for j in range(M):
        s = str(datetime.datetime.strptime('2022-09-16 07:00:00', "%Y-%m-%d %H:%M:%S") + datetime.timedelta(minutes=np.float64(inicio[i][j]) ))
        d = datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S")  + datetime.timedelta(minutes=np.float64(seq_otima[i][j]))
        df.append(dict(Task=str("maq "+str(j+1)), Start=str(s), Finish=str(d), Resource=str("job "+str(i+1))))
        #print(f'Job {i+1} na Máquina {j+1}')
        #print("O tempo de início é: ",s)
        #print("O tempo de término é: ",d)
        #print("\n")
        r = lambda: random.randint(0,255)
        colors.append('#%02X%02X%02X' % (r(),r(),r()))



fig = ff.create_gantt(df, colors = colors, index_col='Resource', title = "FLOW SHOP SCHEDULING - OPTIMUS SOLUTIONS LTDA ",show_colorbar=True, group_tasks=True,showgrid_x =True)    
fig.show()

